In [4]:
from model import *
from data import *
from Block import *

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [6]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5


<ipython-input-6-8261be2be7e1>:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])


2000/2000 [==============================] - ETA: 0s - loss: 0.8680 - accuracy: 0.9135
Epoch 1: loss improved from inf to 0.86801, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 383s 188ms/step - loss: 0.8680 - accuracy: 0.9135
Epoch 2/5
2000/2000 [==============================] - ETA: 0s - loss: 0.1355 - accuracy: 0.9450
Epoch 2: loss improved from 0.86801 to 0.13547, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 375s 188ms/step - loss: 0.1355 - accuracy: 0.9450
Epoch 3/5
2000/2000 [==============================] - ETA: 0s - loss: 0.1036 - accuracy: 0.9570
Epoch 3: loss improved from 0.13547 to 0.10359, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 374s 187ms/step - loss: 0.1036 - accuracy: 0.9570
Epoch 4/5
2000/2000 [==============================] - ETA: 0s - loss: 0.0882 - accuracy: 0.9628
Epoch 4: loss improved from 0.10359 to 0.08825, saving model to unet_membrane.hdf5
2000/2000

You can find my solution for customising the Unet network here. I've created a Block class (in the Block.py file) which creates objects that can be used to build the various recurring sub-parts present in Unet. Then two daughter classes enable the two parts of the network: the encoder part and the decoder part.

In [7]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)

#Here, you can customize your unet network as you desire :
"""To change the Conv2D block by ResNet ones, you need to reconfigure the class Encoder or Decoder by switch the method with the one you desire.
Thanks to that, you can decide, if you want to implement only one ResNet Block (as it is done here) or more."""

input_size = (256, 256, 1)
nb_filters = 64
kernel_size = 3
nb_blocks = 3 #nb of block (Conc Conv MaxPool) that you want to see in your encoder our decoder part

encoder = Encoder(nb_filters=nb_filters, input_size=input_size, kernel_size= kernel_size, nb_blocks= nb_blocks) #Construction of the Encoder part
#model_enc = Model(inputs = encoder.get_inputs(), outputs = encoder.get_outputs()) #Q2.b Creation of the encoder model

decoder = Decoder(nb_blocks=nb_blocks, encoder=encoder) #Construction of the Decodeur part

model = Model(inputs = encoder.get_inputs(), outputs = decoder.get_outputs())

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

#if(pretrained_weights):
#    model.load_weights(pretrained_weights)


model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 256, 256, 1)]        0         []                            
                                                                                                  
 conv2d_48 (Conv2D)          (None, 256, 256, 64)         640       ['input_3[0][0]']             
                                                                                                  
 conv2d_49 (Conv2D)          (None, 256, 256, 64)         36928     ['conv2d_48[0][0]']           
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 128, 128, 64)         0         ['conv2d_49[0][0]']           
 g2D)                                                                                       

<ipython-input-7-f7a57625a497>:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])


Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
2000/2000 [==============================] - ETA: 0s - loss: 0.5700 - accuracy: 0.7812
Epoch 1: loss improved from inf to 0.57003, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 374s 183ms/step - loss: 0.5700 - accuracy: 0.7812
Epoch 2/5
2000/2000 [==============================] - ETA: 0s - loss: 0.5253 - accuracy: 0.7814
Epoch 2: loss improved from 0.57003 to 0.52531, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 366s 183ms/step - loss: 0.5253 - accuracy: 0.7814
Epoch 3/5
2000/2000 [==============================] - ETA: 0s - loss: 0.5251 - accuracy: 0.7814
Epoch 3: loss improved from 0.52531 to 0.52513, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 367s 183ms/step - loss: 0.5251 - accuracy: 0.7814
Epoch 4/5
2000/2000 [==============================] - ETA: 0s - loss: 0.5251 - accuracy: 0.7815
Ep

### Train with npy file

In [9]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2,nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

24/24 [==============================] - ETA: 0s - loss: 0.5279 - accuracy: 0.7793
Epoch 1: loss did not improve from 0.52505
24/24 [==============================] - 5s 197ms/step - loss: 0.5279 - accuracy: 0.7793 - val_loss: 0.5212 - val_accuracy: 0.7845


### test your model and save predicted results

In [10]:
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test",results)

<ipython-input-10-d5059fd62d28>:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(testGene,30,verbose=1)


30/30 [==============================] - 2s 25ms/step


/content/unet/data.py:124: UserWarning: data/membrane/test/0_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)


OSError: ignored